### Project 1

We need to design an dimplement a class that will be used to represent bank accounts.

We want the following functionality and characteristics:
- accounts are uniquely identified by an **account number** (assume it will just be passed in the initializer)
- account holders have a **first** and **last** name
- accounts have an associated **preferred time zone offset** (e.g. -7 for MST)
- **balances** need to be zero or higher, and should not be directly settable.
- but, **deposits** and **withdrawals** can be made (given sufficient funds)
    - if a withdrawal is attempted that would result in nagative funds, the transaction should be declined.
- a **monthly interest rate** exists and is applicable to all accounts **uniformly**. There should be a method that can be called to calculate the interest on the current balance using the current interest rate, and **add it** to the balance.
- each deposit and withdrawal must generate a **confirmation number** composed of:
    - the transaction type: `D` for deposit, and `W` for withdrawal, `I` for interest deposit, and `X` for declined (in which case the balance remains unaffected)
    - the account number
    - the time the transaction was made, using UTC
    - an incrementing number (that increments across all accounts and transactions)
    - for (extreme!) simplicity assume that the transaction id starts at zero (or whatever number you choose) whenever the program starts
    - the confirmation number should be returned from any of the transaction methods (deposit, withdraw, etc)
- create a **method** that, given a confirmation number, returns:
    - the account number, transaction code (D, W, etc), datetime (UTC format), date time (in whatever timezone is specified in te argument, but more human readable), the transaction ID
    - make it so it is a nicely structured object (so can use dotted notation to access these three attributes)
    - I purposefully made it so the desired timezone is passed as an argument. Can you figure out why? (hint: does this method require any information from any instance?)

For example, we may have an account with:
- account number `140568` 
- preferred time zone offset of -7 (MST) 
- an existing balance of `100.00`

Suppose the last transaction ID in the system was `123`, and a deposit is made for `50.00` on `2019-03-15T14:59:00` (UTC) on that account (or `2019-03-15T07:59:00` in account's preferred time zone offset)

The new balance should reflect `150.00` and the confirmation number returned should look something like this:

```D-140568-20190315145900-124```

We also want a method that given the confirmation number returns an object with attributes:
- `result.account_number` --> `140568`
- `result.transaction_code` --> `D`
- `result.transaction_id` --> `124`
- `result.time` --> `2019-03-15 07:59:00 (MST)`
- `result.time_utc` --> `2019-03-15T14:59:00`

Furthermore, if current interest rate is `0.5%`, and the account's balance is `1000.00`, then the result of calling the `deposit_interest` (or whatever name you choose) method, should result in a new transaction and a new balance of `1050.00`. Calling this method should also return a confirmation number.

For simplicty, just use floats, but be aware that for these types of situations you'll probably want to use `Decimal` objects instead of floats.

There are going to be many ways to design something like this, especially since I have not nailed down all the specific requirements, so you'll have to fill the gaps yourself and decide what other things you may want to implement (like is the account number going to be a mutable property, or "read-only" and so on).

See how many different ideas you can use from what we covered in the last section. 

My approach will end up creating two classes: a `TimeZone` class used to store the time zone name and offset definition (in hours and minutes), and a main class called `Account` that will have the following "public" interface:
- initializer with account number, first name, last name, optional preferred time zone, starting balance (defaults to 0)
- a first name property (read/write)
- a last name property (read/write)
- a full name property (computed, read-only)
- a balance property (read-only)
- an interest rate property (class level property)
- deposit, withdraw, pay_interest methods
- parse confirmation code

Class will have additional state and methods, but those will be used for implementation.

You should also remember to test your code! In the solutions I will introduce you to Python's `unittest` package. Even if you skip this project, at least review that video and/or notebook if you are unfamiliar with `unittest`.

In [13]:
from datetime import timedelta, datetime
import numbers, itertools
from collections import namedtuple

## Timezone class

In [14]:
class TimeZone:
    def __init__(self, name, offset_hours, offset_minutes) -> None:
        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Name must be filled')
        
        self._name = str(name).strip() 

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Hour offset must be integer')

        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minute offset must be integer')
        
        if offset_minutes > 59 and offset_minutes < -59:
            raise ValueError('Minutes bust be between -59 and 59')
        
        offset = timedelta(hours=offset_hours, minutes=offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00')        

        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset 
    
    @property
    def name(self):
        return self._name

    def __eq__(self, other):
        return (isinstance(other, TimeZone) and 
                self.name == other.name and 
                self._offset_hours == other._offset_hours and
                self._offset_minutes == other._offset_minutes)
    
    def __repr__(self):
        return (f"TimeZone(name='{self._name}', offset_hours={self._offset_hours}, "
                f"offset_minutes={self._offset_minutes})")

## Account Class

In [15]:
class Account:
    transaction_counter = itertools.count(0)
    _interest_rate = 0.5 # percent
    _transactional_codes = {
        'deposit': 'D', 
        'withdraw': 'W', 
        'interest': 'I',
        'rejected': 'X'
    }

    Confirmation = namedtuple('Confirmation', 'account_number transaction_code transaction_id time_utc time')

    def __init__(self, account_number, first_name, last_name, initial_blance=0, timezone=None):
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name

        if timezone is None:
            timezone = TimeZone('UTC', 0, 0)
        
        self.timezone = timezone

        validate_initial_balance = Account.validate_real_number(value=initial_blance, field_name='Inital Balance', min_value=0)
        self._balance = float(validate_initial_balance)

    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a real number')
        
        if value < 0:
            raise ValueError('Interest rate cannot be negative')
    
        if value > 1:
            raise ValueError('Interest rate must be a percent number')
        
        cls._interest_rate = value

    @property
    def balance(self):
        return self._balance

    @property
    def account_number(self):
        return self._account_number
    
    @property
    def first_name(self): 
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = Account.validate_name(value=value, field='First Name')

    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self._last_name = Account.validate_name(value=value, field='Last Name') 

    @property
    def timezone(self):
        return self._timezone

    @timezone.setter
    def timezone(self, value):
        if not isinstance(value, TimeZone):
            raise ValueError('Time zone must be TimeZone instance or object')
        self._timezone = value 

    def deposit(self, value):
        value = Account.validate_real_number(value=value, field_name='Deposit', min_value=0.01)
        transaction_code = Account._transactional_codes['deposit']
        confirmation_code = self.generate_confirmation_code(transaction_code=transaction_code) 
        self._balance += value
        return confirmation_code
    
    def withdraw(self, value):
        value = Account.validate_real_number(value=value, field_name='withdraw', min_value=0.01)
        transaction_code_key = 'rejected' if self.balance - value < 0 else 'withdraw'
        transaction_code = Account._transactional_codes[transaction_code_key]
        
        confirmation_code = self.generate_confirmation_code(transaction_code=transaction_code)
        
        if transaction_code_key != 'rejected':
            self._balance -= value 
        
        return confirmation_code
    
    def pay_interest(self):
        interest = self.balance * Account.get_interest_rate() / 100 
        confirmation_code = self.generate_confirmation_code(transaction_code=Account._transactional_codes['interest'])
        self._balance += interest
        return confirmation_code
    
    def generate_confirmation_code(self, transaction_code):
        dt_str = datetime.now().strftime('%Y%m%d%H%M%S')
        return f"{transaction_code}-{self.account_number}-{dt_str}-{next(Account.transaction_counter)}"
    
    @staticmethod
    def validate_real_number(value, field_name, min_value=None):
        if not isinstance(value, numbers.Real):
            raise ValueError(f'{str(field_name).capitalize()} value must be a real number')
        
        if min_value is not None and value < min_value:
            raise ValueError(f'{str(field_name).capitalize()} value must be at least {min_value}')
        
        return value
        
    @staticmethod
    def validate_name(value, field):
        if len(str(value).strip()) == 0 or value is None:
            raise ValueError(f'{field} must not be empty')
        return str(value).strip()
    
    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone=None):
        parts = confirmation_code.split('-')
        if len(parts) != 4:
            raise ValueError("Invalid confirmation code")
        
        transaction_code, account_number, raw_utc_str, transaction_id = parts 
        
        try:
            utc_date = datetime.strptime(raw_utc_str, '%Y%m%d%H%M%S') 
        except ValueError as ve:
            raise ValueError('Invalid transaction datetime') from ve 
        
        if preferred_time_zone is None:
            preferred_time_zone = TimeZone('UTC', 0, 0)

        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError('Preferred timezone must be TimeZone object or instance')
        
        dt_preferred = utc_date + preferred_time_zone.offset
        dt_preferred_str = f"{dt_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_time_zone.name})"
        
        return Account.Confirmation(account_number, transaction_code, transaction_id, 
                            utc_date.isoformat(), dt_preferred_str)



In [ ]:
tmz = TimeZone(name='UTC', offset_hours=0, offset_minutes=0)
a1 = Account(account_number=1, first_name='Lorena', last_name='Miranda', timezone=tmz, initial_blance=100)
a2 = Account(123, 'Monty', 'Python')
a3 = Account('A100', 'Eric', 'Idle', timezone=TimeZone(name='UTC-3', offset_hours=-3, offset_minutes=0), initial_blance=100)

## Tests

In [23]:
import unittest

def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(testCaseClass=test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

In [43]:
class TestAccount(unittest.TestCase):
    
    def setUp(self):
        print('Running setup... ')
        self.x = 100

    def tearDown(self):
        print("Running tearDown...")

    def test_create_timezone(self):
        tz = TimeZone('ABC', -1, -30)
        self.assertEqual('ABC', tz.name)
        self.assertEqual(timedelta(hours=-1, minutes=-30), tz.offset)

    def test_timezones_equal(self):
        tz1 = TimeZone('ABC', -1, -30)
        tz2 = TimeZone('ABC', -1, -30)
        self.assertEqual(tz1, tz2)

    def test_timezones_not_equal(self):
        tz1 = TimeZone('ABC', -1, -30)

        test_timezones = (
        TimeZone('UTC-3', -3, 0),
        TimeZone('Udbe', -3, 0),
        TimeZone('ABC', -4, 0)
        )

        for test_tz in test_timezones:
            self.assertNotEqual(tz1, test_tz)


In [44]:
run_tests(TestAccount)

test_create_timezone (__main__.TestAccount) ... ok
test_timezones_equal (__main__.TestAccount) ... ok
test_timezones_not_equal (__main__.TestAccount) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


Running setup... 
Running tearDown...
Running setup... 
Running tearDown...
Running setup... 
Running tearDown...
